# Likelihood comparison

## Compare the likelihood descriptions available in `alldecays`

In [ ]:
import alldecays
from alldecays.fitting.plugins import available_fit_modes

# The following import loads a typical setup procedure.
from higgs_data_setup import data_set, _brs

fits = {}
toys = {}
for fit_mode_name in available_fit_modes:
    # if fit_mode_name in ["Multinomial"]:
    #     continue
    for has_limits in [False, True]:
        key_name = fit_mode_name
        if has_limits:
            key_name = fit_mode_name + "_w_limits"
        fits[key_name] = alldecays.Fit(
            data_set,
            fit_mode_name,
            has_limits=has_limits,
            print_brs_sum_not_1=False,
            raise_invalid_fit_exception=False,
        )
        print(f"{key_name}: {fits[key_name].Minuit.fval} is the fit minimum.")
        toys[key_name] = fits[key_name].fill_toys(n_toys=10)
fig_values = alldecays.plotting.compare_values(fits, shift_x=True)

In [ ]:
alldecays.plotting.compare_errors_only(fits, shift_x=True)
alldecays.plotting.compare_errors_only(fits);

In [ ]:
alldecays.plotting.compare_errors_only(
    fits, shift_x=True, as_relative_coupling_error=False
);

In [ ]:
fits["Poisson"].Minuit.errors

## Scaling of the (statistical) error with luminosity

In [ ]:
lumi_fits = {}
tmp_lumi = data_set.luminosity_ifb
for lumi in [200, 500, 1000, 2000, 5000]:
    data_set.luminosity_ifb = lumi
    lumi_fits[str(lumi)] = alldecays.Fit(data_set)
data_set.luminosity_ifb = tmp_lumi
ax = alldecays.plotting.compare_errors_only(lumi_fits)
ax.legend().set_title("Integrated\n luminosity:");

## Add external measurements

In [ ]:
from alldecays.plotting.util import FitParameters
from higgs_couplings_external import coupling_projections

compare_fits = dict(fits)
for k, error_dict in coupling_projections.items():
    names = list(error_dict.keys())
    values = [_brs[n] for n in names]
    # coupling_projections is citing relative coupling errors.
    # `* 2` to move to BRs. `* br` to move to total errors.
    errors = [error_dict[n] * 2 * _brs[n] for n in names]
    compare_fits[k] = FitParameters(
        names=names,
        values=values,
        errors=errors,
        covariance=None,
        starting_values=None,
    )
fig_ext = alldecays.plotting.compare_errors_only(compare_fits)